In [20]:
from keras.applications import inception_v3
import keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

K.set_learning_phase(0)
model = inception_v3.InceptionV3(weights='imagenet',include_top= False)

c:\Users\natsc\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [21]:
layers_contributions = {
    'mixed2' : 0.2,
    'mixed3' : 3.,
    'mixed4' : 2.,
    'mixed5' : 1.5,}

In [22]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layers_contributions:
    coeff = layers_contributions[layer_name]
    activation = layer_dict[layer_name].output

    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :1])) / scaling


In [39]:
import numpy as np

In [26]:
dream = model.input
grads = K.gradients(loss, dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)
outputs = [loss,grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_value = outs[1]
    return loss_value, grad_value

def gradient_ascent(x, iterations,step,max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at',i,':',loss_value)
        x += step * grad_values
    return x 

In [67]:
import keras
import imageio

In [68]:
import scipy
from keras.preprocessing.image import image_utils 

def resize_img(img, size):
    img = np.copy(img)
    factor = (1,
            float(size[0]) / img.shape[1],
            float(size[1]) / img.shape[2],
            1)
    return scipy.ndimage.zoom(img, factor, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    imageio.imwrite(fname,pil_img)

def preproccess_image(image_path):
    img = image_utils.load_img(image_path)
    img = image_utils.img_to_array(img)
    img = np.expand_dims(img, axis=  0)
    img = inception_v3.preprocess_input(img)
    return img
def deprocess_image(x):
    if K.image_data_format() == 'channel_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1,2,0))
    else:
        x = x.reshape((x.shape[1], x.shape[2],3))
    x /= 2,
    x += 0.5
    x *= 255.
    x = np.clip(x,0,255).astype('uint8')
    return x

In [72]:



step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10

base_image_path = r'C:\Users\natsc\Desktop\machine_learning\deeplearningwithpython\img\city_image.jpg'

img = preproccess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
        for dim in original_shape])
    successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img,successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                        iterations=iterations,
                        step=step,
                        max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale' + str(shape) + '.png')


save_img(img, fname='final_dream.png')


Processing image shape (304, 459)
...Loss value at 0 : 0.0031241947
...Loss value at 1 : 0.012584522
...Loss value at 2 : 0.027620535
...Loss value at 3 : 0.045795303
...Loss value at 4 : 0.064832695
...Loss value at 5 : 0.08225382
...Loss value at 6 : 0.101717114
...Loss value at 7 : 0.12062002
...Loss value at 8 : 0.13635248
...Loss value at 9 : 0.15535337
...Loss value at 10 : 0.17133059
...Loss value at 11 : 0.18912937
...Loss value at 12 : 0.20299397
...Loss value at 13 : 0.21937685
...Loss value at 14 : 0.23451394
...Loss value at 15 : 0.2485198
...Loss value at 16 : 0.2642999
...Loss value at 17 : 0.2795529
...Loss value at 18 : 0.2924939
...Loss value at 19 : 0.30471438
Processing image shape (426, 642)
...Loss value at 0 : 0.029338144
...Loss value at 1 : 0.06388867
...Loss value at 2 : 0.0922939
...Loss value at 3 : 0.11656547
...Loss value at 4 : 0.13966724
...Loss value at 5 : 0.16231743
...Loss value at 6 : 0.18345
...Loss value at 7 : 0.20384581
...Loss value at 8 : 0.224